## **Setup**

In [1]:
import tensorflow as tf
from scipy.spatial.distance import cdist
from scipy.special import softmax

from tqdm.notebook import tqdm
import numpy as np
import os

In [2]:
DEPTH_MULTIPLIER_PREFIX = '0'
DEPTH_MULTIPLIER = 25
IMAGE_SIZE = 128
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
BATCH_SIZE = 100
NUM_EPOCHS = 20

In [3]:
#enable jupyter widgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## **Load model**

In [4]:
model_name = 'mobilenet_v1_'+ DEPTH_MULTIPLIER_PREFIX + str(DEPTH_MULTIPLIER)+'_'+str(IMAGE_SIZE)+'_epochs_'+str(NUM_EPOCHS)
model_path = 'models\\' + model_name
model = tf.keras.models.load_model(model_path)

In [5]:
#init feature extractor layer
fel = model.layers[0]
print(fel)

## **Initialize paths**

In [6]:
#root path
root = "datasets\\data-cifar-10"
ROOT_TRAIN_SUBDIR = "train"
ROOT_VAL_SUBDIR = "validation"
root_training_set_path = os.path.join(root, ROOT_TRAIN_SUBDIR)
root_validation_set_path = os.path.join(root, ROOT_VAL_SUBDIR)
print(root_training_set_path)
print(root_validation_set_path)

datasets\data-cifar-10\train
datasets\data-cifar-10\validation


In [7]:
# OOD data paths
root_ood_data_path = "datasets\\svhn_test"
print(root_ood_data_path)
list_files = os.listdir(root_ood_data_path)
total_ood_images = len(list_files)
print(total_ood_images)

datasets\svhn_test
13068


## **Initialize folders**

In [8]:
#init names
OUTPUT_FOLDER_NAME = "outputs"
DATA_NAME = "CIFAR_10"
OOD_DATA_NAME = "svhn"

In [9]:
#init features folders
FEATURES_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "features")
FEATURES_DATA_FOLDER = os.path.join(FEATURES_FOLDER, DATA_NAME)
OOD_FEATURES_DATA_FOLDER = os.path.join(FEATURES_FOLDER, OOD_DATA_NAME)
    
if(not os.path.exists(FEATURES_DATA_FOLDER)):
    os.makedirs(FEATURES_DATA_FOLDER)

if(not os.path.exists(OOD_FEATURES_DATA_FOLDER)):
    os.makedirs(OOD_FEATURES_DATA_FOLDER)
    
#init distances folders
DISTS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "dists")
DISTS_DATA_FOLDER = os.path.join(DISTS_FOLDER, DATA_NAME)
OOD_DISTS_DATA_FOLDER = os.path.join(DISTS_FOLDER, OOD_DATA_NAME)

if(not os.path.exists(DISTS_DATA_FOLDER)):
    os.makedirs(DISTS_DATA_FOLDER)

if(not os.path.exists(OOD_DISTS_DATA_FOLDER)):
    os.makedirs(OOD_DISTS_DATA_FOLDER)
    
#init closest class folders
CLOSEST_CLASS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "closest_classes")
CLOSEST_CLASS_DATA_FOLDER = os.path.join(CLOSEST_CLASS_FOLDER, DATA_NAME)
OOD_CLOSEST_CLASS_DATA_FOLDER = os.path.join(CLOSEST_CLASS_FOLDER, OOD_DATA_NAME)

if(not os.path.exists(CLOSEST_CLASS_DATA_FOLDER)):
    os.makedirs(CLOSEST_CLASS_DATA_FOLDER)

if(not os.path.exists(OOD_CLOSEST_CLASS_DATA_FOLDER)):
    os.makedirs(OOD_CLOSEST_CLASS_DATA_FOLDER)
    
#init probabilities folders
PROBABILITIES_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "probabilities")
PROBABILITIES_DATA_FOLDER = os.path.join(PROBABILITIES_FOLDER, DATA_NAME)
OOD_PROBABILITIES_DATA_FOLDER = os.path.join(PROBABILITIES_FOLDER, OOD_DATA_NAME)

if(not os.path.exists(PROBABILITIES_DATA_FOLDER)):
    os.makedirs(PROBABILITIES_DATA_FOLDER)
    
if(not os.path.exists(OOD_PROBABILITIES_DATA_FOLDER)):
    os.makedirs(OOD_PROBABILITIES_DATA_FOLDER)

#init labels folders
LABELS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "labels")
LABELS_DATA_FOLDER = os.path.join(LABELS_FOLDER, DATA_NAME)

if(not os.path.exists(LABELS_DATA_FOLDER)):
    os.makedirs(LABELS_DATA_FOLDER)
    
#init means folders
MEANS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "means")
MEANS_DATA_FOLDER = os.path.join(MEANS_FOLDER, DATA_NAME)

if(not os.path.exists(MEANS_DATA_FOLDER)):
    os.makedirs(MEANS_DATA_FOLDER)
    
#init radius folders
RADIUS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "radius")
RADIUS_DATA_FOLDER = os.path.join(RADIUS_FOLDER, DATA_NAME)

if(not os.path.exists(RADIUS_DATA_FOLDER)):
    os.makedirs(RADIUS_DATA_FOLDER)

## **Initialize parameters**

In [10]:
TOTAL_TRAIN_DATA_COUNT = 50000
TEMPERATURE = 100
ALPHA = 0.5

## **Init generator and class names**

In [11]:
root_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [12]:
root_image_data_train = root_image_generator.flow_from_directory(str(root_training_set_path), target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE)
root_image_data_validation = root_image_generator.flow_from_directory(str(root_validation_set_path), target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE )

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [13]:
for root_image_batch, root_label_batch in root_image_data_train:
    print("Image batch shape: ", root_image_batch.shape)
    print("Label batch shape: ", root_label_batch.shape)
    break

Image batch shape:  (100, 128, 128, 3)
Label batch shape:  (100, 10)


In [14]:
root_class_names = sorted(root_image_data_train.class_indices.items(), key=lambda pair:pair[1])
root_class_names = np.array([key.title() for key, value in root_class_names])
root_class_names

array(['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog',
       'Horse', 'Ship', 'Truck'], dtype='<U10')

In [15]:
NUM_CLASSES = root_class_names.size
print("No. of classes - " + str(NUM_CLASSES))

No. of classes - 10


## **Generating features**

__1. CIFAR 10 train features__

In [16]:
t = tqdm(root_class_names)
for className in t:
    cnt=0
    xx=[]
    train_set_features = np.empty((0, 256), float)
    sub_path = os.path.join(root_training_set_path, className)
    for filename in os.listdir(sub_path):
        t.set_description("Generating features for class - " + className)
        cnt = cnt + 1
        full_path = os.path.join(sub_path, filename)
        img = tf.keras.preprocessing.image.load_img(full_path, target_size=[128, 128])
        x = tf.keras.preprocessing.image.img_to_array(img)/255.0 #resize image
        xx.append(x)
        if( cnt%500 != 0):
            continue
            
        #assemble features
        xx = np.asarray(xx)
        train_img_features = np.asarray(fel(xx))
        train_set_features = np.vstack((train_set_features, train_img_features))
        xx=[]

    train_set_features = np.asarray(train_set_features)
    save_name = "{0}_train_features_with_{1}_samples.npy".format(className, train_set_features.shape[0])
    save_location =  os.path.join(FEATURES_DATA_FOLDER, save_name)
    np.save(save_location, train_set_features)
    print("Saved generated features for " + className + " at " + save_location + " having shape - " + str(train_set_features.shape))

Saved generated features for Airplane at outputs\features\CIFAR_10\Airplane_train_features_with_5000_samples.npy having shape - (5000, 256)
Saved generated features for Automobile at outputs\features\CIFAR_10\Automobile_train_features_with_5000_samples.npy having shape - (5000, 256)
Saved generated features for Bird at outputs\features\CIFAR_10\Bird_train_features_with_5000_samples.npy having shape - (5000, 256)
Saved generated features for Cat at outputs\features\CIFAR_10\Cat_train_features_with_5000_samples.npy having shape - (5000, 256)
Saved generated features for Deer at outputs\features\CIFAR_10\Deer_train_features_with_5000_samples.npy having shape - (5000, 256)
Saved generated features for Dog at outputs\features\CIFAR_10\Dog_train_features_with_5000_samples.npy having shape - (5000, 256)
Saved generated features for Frog at outputs\features\CIFAR_10\Frog_train_features_with_5000_samples.npy having shape - (5000, 256)
Saved generated features for Horse at outputs\features\CIFAR

__2. CIFAR 10 validation features__

In [17]:
t = tqdm(root_class_names)
for className in t:
    cnt=0
    xx=[]
    validation_set_features = np.empty((0, 256), float)
    sub_path = os.path.join(root_validation_set_path, className)
    for filename in os.listdir(sub_path):
        t.set_description("Generating features for class - " + className)
        cnt = cnt + 1
        full_path = os.path.join(sub_path, filename)
        img = tf.keras.preprocessing.image.load_img(full_path, target_size=[128, 128])
        x = tf.keras.preprocessing.image.img_to_array(img)/255.0 #resize image
        xx.append(x)
        if( cnt%500 != 0):
            continue
            
        #assemble features
        xx = np.asarray(xx)
        validation_img_features = np.asarray(fel(xx))
        validation_set_features = np.vstack((validation_set_features, validation_img_features))
        xx=[]

    validation_set_features = np.asarray(validation_set_features)
    save_name = "{0}_validation_features_with_{1}_samples.npy".format(className, validation_set_features.shape[0])
    save_location =  os.path.join(FEATURES_DATA_FOLDER, save_name)
    np.save(save_location, validation_set_features)
    print("Saved generated features for " + className + " at " + save_location + " having shape - " + str(validation_set_features.shape))

Saved generated features for Airplane at outputs\features\CIFAR_10\Airplane_validation_features_with_1000_samples.npy having shape - (1000, 256)
Saved generated features for Automobile at outputs\features\CIFAR_10\Automobile_validation_features_with_1000_samples.npy having shape - (1000, 256)
Saved generated features for Bird at outputs\features\CIFAR_10\Bird_validation_features_with_1000_samples.npy having shape - (1000, 256)
Saved generated features for Cat at outputs\features\CIFAR_10\Cat_validation_features_with_1000_samples.npy having shape - (1000, 256)
Saved generated features for Deer at outputs\features\CIFAR_10\Deer_validation_features_with_1000_samples.npy having shape - (1000, 256)
Saved generated features for Dog at outputs\features\CIFAR_10\Dog_validation_features_with_1000_samples.npy having shape - (1000, 256)
Saved generated features for Frog at outputs\features\CIFAR_10\Frog_validation_features_with_1000_samples.npy having shape - (1000, 256)
Saved generated features 

__3. Svhn test features__

In [18]:
t = tqdm(list_files, desc="Generating features for svhn dataset")
cnt=0
xx=[]
ood_set_features = np.empty((0, 256), float)
for filename in t:
    cnt = cnt + 1
    full_path = os.path.join(root_ood_data_path, filename)
    img = tf.keras.preprocessing.image.load_img(full_path, target_size=[128, 128])
    x = tf.keras.preprocessing.image.img_to_array(img)/255.0 #resize image
    xx.append(x)
    if( cnt%500 !=0 and cnt%total_ood_images!=0):
        continue

    #assemble features
    xx = np.asarray(xx)
    ood_img_features = np.asarray(fel(xx))
    ood_set_features = np.vstack((ood_set_features, ood_img_features))
    xx=[]

ood_set_features = np.asarray(ood_set_features)
save_name = "{0}_features_with_{1}_samples.npy".format(OOD_DATA_NAME, ood_set_features.shape[0])
save_location =  os.path.join(OOD_FEATURES_DATA_FOLDER, save_name)
np.save(save_location, ood_set_features)
print("Saved generated features for " + OOD_DATA_NAME + " at " + save_location + " having shape - " + str(ood_set_features.shape))


Saved generated features for svhn at outputs\features\svhn\svhn_features_with_13068_samples.npy having shape - (13068, 256)


## **Generating class means**

In [19]:
class_means=[]
t = tqdm(root_class_names)
for className in t:
    t.set_description("Calculating mean for class - " + className)
    save_name = "{0}_train_features_with_{1}_samples.npy".format(className, 5000)
    class_features_path = os.path.join(FEATURES_DATA_FOLDER, save_name)
    class_features = np.load(class_features_path)
    cMean = np.mean(class_features, axis=0)
    class_means.append(cMean)
class_means = np.asarray(class_means)
save_name = "CIFAR_10_class_means.npy"
save_location =  os.path.join(MEANS_DATA_FOLDER, save_name)
np.save(save_location, class_means)
print("Saved generated means at " + save_location + " having shape - " + str(class_means.shape))


Saved generated means at outputs\means\CIFAR_10\CIFAR_10_class_means.npy having shape - (10, 256)


## **Generating class radius**

In [20]:
PER_CLASS_SIZE = 5000
FRACTION_TO_COVER = 0.95
MODIFIED_PER_CLASS_SIZE = int(PER_CLASS_SIZE*FRACTION_TO_COVER)

In [21]:
#load class means, if not already loaded
save_name = "CIFAR_10_class_means.npy"
save_location =  os.path.join(MEANS_DATA_FOLDER, save_name)
class_means = np.load(save_location)

In [22]:
class_radii=[]
t = tqdm(root_class_names)
cnt = 0
for className in t:
    t.set_description("Calculating radius for class - " + className)
    save_name = "{0}_train_features_with_{1}_samples.npy".format(className, 5000)
    save_location = os.path.join(FEATURES_DATA_FOLDER, save_name)
    class_features = np.load(save_location)
    radii = np.sort(np.ndarray.flatten(cdist(class_features, np.expand_dims(class_means[cnt], axis=0), metric='cosine')))
    class_radii.append(radii[MODIFIED_PER_CLASS_SIZE])
    cnt = cnt + 1
class_radii = np.asarray(class_radii)
save_name = "CIFAR_10_class_radii.npy"
save_location =  os.path.join(RADIUS_DATA_FOLDER, save_name)
np.save(save_location, class_radii)
print("Saved generated radii at " + save_location + " having shape - " + str(class_radii.shape))


Saved generated radii at outputs\radius\CIFAR_10\CIFAR_10_class_radii.npy having shape - (10,)


## **Calculate distances & closest classes**

__1. CIFAR 10 train distances & classes__

In [23]:
t = tqdm(root_class_names)
for className in t:
    train_dists = []
    train_closest_classes = []
    t.set_description("Calculating distance for class - " + className)
    save_name = "{0}_train_features_with_{1}_samples.npy".format(className, 5000)
    save_location = os.path.join(FEATURES_DATA_FOLDER, save_name)
    train_set_features = np.load(save_location)
    for image_feature in train_set_features:
        d = cdist(class_means, np.expand_dims(image_feature, axis=0), metric='cosine')
        idx = np.argmin(d)
        d = d[idx][0]
        train_closest_classes.append(idx)
        train_dists.append(d)
        
    train_dists = np.asarray(train_dists)
    save_name = "{0}_train_distances_with_{1}_samples.npy".format(className, train_dists.shape[0])
    save_location =  os.path.join(DISTS_DATA_FOLDER, save_name)
    np.save(save_location, train_dists)
    print("Saved distances for " + className + " at " + save_location + " having shape - " + str(train_dists.shape))
    
    train_closest_classes = np.asarray(train_closest_classes)
    save_name = "{0}_train_closest_classes_with_{1}_samples.npy".format(className, train_closest_classes.shape[0])
    save_location =  os.path.join(CLOSEST_CLASS_DATA_FOLDER, save_name)
    np.save(save_location, train_closest_classes)
    print("Saved closest classes for " + className + " at " + save_location + " having shape - " + str(train_closest_classes.shape))

Saved distances for Airplane at outputs\dists\CIFAR_10\Airplane_train_distances_with_5000_samples.npy having shape - (5000,)
Saved closest classes for Airplane at outputs\closest_classes\CIFAR_10\Airplane_train_closest_classes_with_5000_samples.npy having shape - (5000,)
Saved distances for Automobile at outputs\dists\CIFAR_10\Automobile_train_distances_with_5000_samples.npy having shape - (5000,)
Saved closest classes for Automobile at outputs\closest_classes\CIFAR_10\Automobile_train_closest_classes_with_5000_samples.npy having shape - (5000,)
Saved distances for Bird at outputs\dists\CIFAR_10\Bird_train_distances_with_5000_samples.npy having shape - (5000,)
Saved closest classes for Bird at outputs\closest_classes\CIFAR_10\Bird_train_closest_classes_with_5000_samples.npy having shape - (5000,)
Saved distances for Cat at outputs\dists\CIFAR_10\Cat_train_distances_with_5000_samples.npy having shape - (5000,)
Saved closest classes for Cat at outputs\closest_classes\CIFAR_10\Cat_train_c

__2. CIFAR 10 validation distances & classes__

In [24]:
t = tqdm(root_class_names)
for className in t:
    validation_dists = []
    validation_closest_classes = []
    t.set_description("Calculating distance for class - " + className)
    save_name = "{0}_validation_features_with_{1}_samples.npy".format(className, 1000)
    save_location = os.path.join(FEATURES_DATA_FOLDER, save_name)
    validation_set_features = np.load(save_location)
    for image_feature in validation_set_features:
        d = cdist(class_means, np.expand_dims(image_feature, axis=0), metric='cosine')
        idx = np.argmin(d)
        d = d[idx][0]
        validation_closest_classes.append(idx)
        validation_dists.append(d)
        
    validation_dists = np.asarray(validation_dists)
    save_name = "{0}_validation_distances_with_{1}_samples.npy".format(className, validation_dists.shape[0])
    save_location =  os.path.join(DISTS_DATA_FOLDER, save_name)
    np.save(save_location, validation_dists)
    print("Saved distances for " + className + " at " + save_location + " having shape - " + str(validation_dists.shape))
    
    validation_closest_classes = np.asarray(validation_closest_classes)
    save_name = "{0}_validation_closest_classes_with_{1}_samples.npy".format(className, validation_closest_classes.shape[0])
    save_location =  os.path.join(CLOSEST_CLASS_DATA_FOLDER, save_name)
    np.save(save_location, validation_closest_classes)
    print("Saved closest classes for " + className + " at " + save_location + " having shape - " + str(validation_closest_classes.shape))

Saved distances for Airplane at outputs\dists\CIFAR_10\Airplane_validation_distances_with_1000_samples.npy having shape - (1000,)
Saved closest classes for Airplane at outputs\closest_classes\CIFAR_10\Airplane_validation_closest_classes_with_1000_samples.npy having shape - (1000,)
Saved distances for Automobile at outputs\dists\CIFAR_10\Automobile_validation_distances_with_1000_samples.npy having shape - (1000,)
Saved closest classes for Automobile at outputs\closest_classes\CIFAR_10\Automobile_validation_closest_classes_with_1000_samples.npy having shape - (1000,)
Saved distances for Bird at outputs\dists\CIFAR_10\Bird_validation_distances_with_1000_samples.npy having shape - (1000,)
Saved closest classes for Bird at outputs\closest_classes\CIFAR_10\Bird_validation_closest_classes_with_1000_samples.npy having shape - (1000,)
Saved distances for Cat at outputs\dists\CIFAR_10\Cat_validation_distances_with_1000_samples.npy having shape - (1000,)
Saved closest classes for Cat at outputs\c

__3. Svhn test distances & classes__

In [25]:
ood_dists = []
ood_closest_classes = []
save_name = "{0}_features_with_{1}_samples.npy".format(OOD_DATA_NAME, 13068)
save_location = os.path.join(OOD_FEATURES_DATA_FOLDER, save_name)
ood_set_features = np.load(save_location)
for image_feature in ood_set_features:
    d = cdist(class_means, np.expand_dims(image_feature, axis=0), metric='cosine')
    idx = np.argmin(d)
    d = d[idx][0]
    ood_closest_classes.append(idx)
    ood_dists.append(d)

ood_dists = np.asarray(ood_dists)
save_name = "{0}_distances_with_{1}_samples.npy".format(OOD_DATA_NAME, ood_dists.shape[0])
save_location =  os.path.join(OOD_DISTS_DATA_FOLDER, save_name)
np.save(save_location, ood_dists)
print("Saved distances for " + OOD_DATA_NAME + " at " + save_location + " having shape - " + str(ood_dists.shape))

ood_closest_classes = np.asarray(ood_closest_classes)
save_name = "{0}_closest_classes_with_{1}_samples.npy".format(OOD_DATA_NAME, ood_closest_classes.shape[0])
save_location =  os.path.join(OOD_CLOSEST_CLASS_DATA_FOLDER, save_name)
np.save(save_location, ood_closest_classes)
print("Saved closest classes for " + OOD_DATA_NAME + " at " + save_location + " having shape - " + str(ood_closest_classes.shape))

Saved distances for svhn at outputs\dists\svhn\svhn_distances_with_13068_samples.npy having shape - (13068,)
Saved closest classes for svhn at outputs\closest_classes\svhn\svhn_closest_classes_with_13068_samples.npy having shape - (13068,)


## **Calculate probabilities**

__1.CIFAR 10 train probabilities__

In [27]:
t = tqdm(root_class_names)
for className in t:
    cnt=0
    xx=[]
    train_set_probabilities = np.empty((0,), float)
    train_set_scaled_probabilities = np.empty((0, ), float)
    sub_path = os.path.join(root_training_set_path, className)
    
    #load train dists for the class
    save_name = "{0}_train_distances_with_{1}_samples.npy".format(className, 5000)
    save_location = os.path.join(DISTS_DATA_FOLDER, save_name)
    train_dists = np.load(save_location)
    
    for filename in os.listdir(sub_path):
        t.set_description("Generating probabilities for class - " + className)
        cnt = cnt + 1
        full_path = os.path.join(sub_path, filename)
        img = tf.keras.preprocessing.image.load_img(full_path, target_size=[128, 128])
        x = tf.keras.preprocessing.image.img_to_array(img)/255.0 #resize image
        xx.append(x)
        if( cnt%500 != 0):
            continue
            
            
       #make predictions
        xx = np.asarray(xx)
        probabilities_batch = model.predict(xx)
        probabilities_scaled_batch = np.empty((0,10), float)
        for i in range(500):
            probabilities_scaled_batch = np.vstack((probabilities_scaled_batch, softmax(probabilities_batch[i]/TEMPERATURE*train_dists[cnt-1])))
        probabilities_batch = np.max(probabilities_batch, axis=-1)
        probabilities_scaled_batch = np.max(probabilities_scaled_batch, axis=-1)
        
        train_set_probabilities = np.concatenate((train_set_probabilities, probabilities_batch))
        train_set_scaled_probabilities = np.concatenate((train_set_scaled_probabilities, probabilities_scaled_batch))

        #re-init vars
        xx=[]

    train_set_probabilities = np.asarray(train_set_probabilities)
    save_name = "{0}_train_probabilities_with_{1}_samples.npy".format(className, train_set_probabilities.shape[0])
    save_location =  os.path.join(PROBABILITIES_DATA_FOLDER, save_name)
    np.save(save_location, train_set_probabilities)
    print("Saved generated probabilities for " + className + " at " + save_location + " having shape - " + str(train_set_probabilities.shape))
    
    train_set_scaled_probabilities = np.asarray(train_set_scaled_probabilities)
    save_name = "{0}_train_scaled_probabilities_with_{1}_samples.npy".format(className, train_set_scaled_probabilities.shape[0])
    save_location =  os.path.join(PROBABILITIES_DATA_FOLDER, save_name)
    np.save(save_location, train_set_scaled_probabilities)
    print("Saved generated scaled probabilities for " + className + " at " + save_location + " having shape - " + str(train_set_scaled_probabilities.shape))

Saved generated probabilities for Airplane at outputs\probabilities\CIFAR_10\Airplane_train_probabilities_with_5000_samples.npy having shape - (5000,)
Saved generated scaled probabilities for Airplane at outputs\probabilities\CIFAR_10\Airplane_train_scaled_probabilities_with_5000_samples.npy having shape - (5000,)
Saved generated probabilities for Automobile at outputs\probabilities\CIFAR_10\Automobile_train_probabilities_with_5000_samples.npy having shape - (5000,)
Saved generated scaled probabilities for Automobile at outputs\probabilities\CIFAR_10\Automobile_train_scaled_probabilities_with_5000_samples.npy having shape - (5000,)
Saved generated probabilities for Bird at outputs\probabilities\CIFAR_10\Bird_train_probabilities_with_5000_samples.npy having shape - (5000,)
Saved generated scaled probabilities for Bird at outputs\probabilities\CIFAR_10\Bird_train_scaled_probabilities_with_5000_samples.npy having shape - (5000,)
Saved generated probabilities for Cat at outputs\probabiliti